In [1]:
import math
import re 
import numpy as np
Tra_x   = [[1.,0.,0.,1.],
                    [0.,1.,0.,0.],
                    [0.,0.,1.,0.],
                    [0.,0.,0.,1.]]


In [2]:
print(Tra_x[0][3],"\n","klaza")

1.0 
 klaza


In [3]:
joint = 3
def forward_kin(q):
    DH_table = ([[0,0,0.3,0.],
                [0.16,math.pi/2,0,0.],
                [0.35,0.,0.,0.]])
    P = [1,1,1]
    H = np.identity(4)

    H3_e = [[0.,0.,1.,0.16],
                    [1.,0.,0.,0.],
                    [0.,-1.,0.,0.],
                    [0.,0.,0.,1.]]

    for i in range(0,joint):
        Tx = DH_table[i][0] 
        Rx = DH_table[i][1] 
        Tz = DH_table[i][2] 
        Rz = DH_table[i][3] 
        Tra_x   = [[1.,0.,0.,Tx],
                [0.,1.,0.,0.],
                [0.,0.,1.,0.],
                [0.,0.,0.,1.]]

        Rot_x   = [[1.,0.,0.,0.],
                [0.,math.cos(Rx),-1*math.sin(Rx),0.],
                [0.,math.sin(Rx),math.cos(Rx),0.],
                [0.,0.,0.,1.]]
                            
        Tra_z   = [[1.,0.,0.,0.],
                [0.,1.,0.,0.],
                [0.,0.,1.,Tz],
                [0.,0.,0.,1.]]
        Rot_z   = [[math.cos(Rz),-1*math.sin(Rz),0.,0.],
                [math.sin(Rz),math.cos(Rz),0.,0.],
                [0.,0.,1.,0.],
                [0.,0.,0.,1.]]

        if P[i] == 1:
            # print(q[i])
            Rot_z_q   = [[math.cos(q[i]),-1*math.sin(q[i]),0.,0.],
                        [math.sin(q[i]),math.cos(q[i]),0.,0.],
                        [0.,0.,1.,0.],
                        [0.,0.,0.,1.]]
            Hj = Rot_z_q
        else:
            Tra_z_q   = [[1.,0.,0.,0.],
                        [0.,1.,0.,0.],
                        [0.,0.,1.,q[i]],
                        [0.,0.,0.,1.]]
            Hj = Tra_z_q

        H = H @ Tra_x @ Rot_x @ Tra_z @ Rot_z @ Hj
        # print(H)
    H0_e = H@H3_e
#     print(Tra_x,"\n",Rot_x,"\n",Tra_z,"\n",Rot_z)
    return H0_e

def forward_kinematics(q):
    joint_config = q
    n_joint = 3
    DH_pam = [[0 ,0 ,0.3 ,0],
              [0.16 ,math.pi/2 ,0 ,0],
              [0.35 ,0 ,0 ,0]]

    H = np.identity(4)

    H_end_eff = [[0 , 0 ,1 ,0.16],
                 [1 , 0 ,0 ,0],
                 [0 ,1 ,0 ,0],
                 [0 , 0 ,0 ,1]]

    for i in range(joint):
        rad = joint_config[i]
        c ,s = np.cos(rad), np.sin(rad)
        c_x ,s_x = np.cos(DH_pam[i][1]), np.sin(DH_pam[i][1])
        c_z ,s_z = np.cos(DH_pam[i][3]), np.sin(DH_pam[i][3])

        T_x =  [[1 ,0 ,0 ,DH_pam[i][0]],               
                [0 ,1 ,0 ,0],
                [0 ,0, 1 ,0],
                [0 ,0 ,0 ,1]]

        R_x =  [[1 ,   0 ,   0 ,0],           
                [0 , c_x ,-s_x ,0],
                [0 , s_x , c_x ,0],
                [0 ,   0 ,   0 ,1]]

        T_z =  [[1 ,0 ,0 ,0],
                [0 ,1 ,0 ,0],
                [0 ,0 ,1 ,DH_pam[i][2]],
                [0 ,0 ,0 ,1]]

        R_z =  [[c_z ,-s_z ,0 ,0],
                [s_z , c_z ,0 ,0],
                [0   ,   0 ,1 ,0],
                [0   ,   0 ,0 ,1]]

        H_j =  [[c ,-s ,0 ,0],
                [s , c ,0 ,0],
                [0 , 0 ,1 ,0],
                [0 , 0 ,0 ,1]]

        H = H @ T_x @ R_x @ T_z @ R_z @ H_j      
        # print(H)
    H = H@H_end_eff
#     print(T_x,"\n",R_x,"\n",T_z,"\n",T_z)

    return H

In [4]:
A = forward_kin([3.14,1.57,1.57])
b = forward_kinematics([3.14,1.57,1.57])
print([A[0][3]],A[1][3],A[2][3])
print([b[0][3]],b[1][3],b[2][3])

[-0.00027891691860720824] 4.4421840724579147e-07 0.65025471349278
[-0.00027891691860720824] 4.4421840724579147e-07 0.65025471349278


In [31]:
import math
import numpy as np

# 2D RRIK
def RRIK(x,y,F,L1,L2):
    c2 = (x**2+y**2-L1**2-L2**2)/(2*L1*L2)
    if -1 <= c2 and c2 <=1 :
        s2 = F*math.sqrt(1-c2**2)
        q3 = math.atan2(y,x)-math.atan2(L2*s2,L1+L2*c2)
    else:
        flag = False
        return [0. ,0. ,0.],flag
    q2 = math.atan2(s2,c2)
    print(c2)
    return [q2,q3]

# 3D RRRIK
def inverse_kin(point,gamma,h1,L1,L2,L3):
    flag = True
    x = point[0]
    y = point[1]
    z = point[2]

    Fq1 = gamma[0]
    Fq23 = gamma[1]
    
    # q1
    r = Fq1*(math.sqrt(x**2+y**2))
    c1 = x/r
    s1 = y/r
    q1 = math.atan2(y/Fq1,x/Fq1)
    # q23
    Q = RRIK(r-L1,z-h1,Fq23,L2,L3)

    c_3 = ((Fq1*((x**2+y**2)**0.5) - L1)**2 + (z-h1)**2 - L2**2 - L3**2)/(2*L2*L3)
    
    print(c_3)
    if Q == ([0. ,0. ,0.],False):
        return [0. ,0. ,0.],False
    else:
        return [q1,Q[0],Q[1]],flag


In [34]:
# point = [A[0][3],A[1][3],A[2][3]]
point = [-0.00027891691860720824,4.4421840724579147e-07,0.65025471349278]
gamma = [1,-1]
gg = inverse_kin(point,gamma,0.3,0.16,0.35,0.16)
print(gg)

0.0007963267107331696
0.0007963267107331696
([3.140000000000077, -1.57, 2.4272805980561225], True)
